# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
N. Wang  ->  N. Wang  |  ['N. Wang']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
T. Henning  ->  T. Henning  |  ['T. Henning']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
F. Walter  ->  F. Walter  |  ['F. Walter']


Arxiv has 66 new papers today
          10 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.02035


extracting tarball to tmp_2504.02035...

 done.
Retrieving document from  https://arxiv.org/e-print/2504.02071


extracting tarball to tmp_2504.02071...

...

✔ → 26849:\section{\label{sec:modeling} Modeling the Observed Emission}
  ↳ 46915:\section{\label{sec:discussion} Results and Discussion}


✔ → 46915:\section{\label{sec:discussion} Results and Discussion}
  ↳ 80192:\section{\label{sec:summary} Summary}
✔ → 80192:\section{\label{sec:summary} Summary}
  ↳ 85986:\section{\label{app:CO abs} CO Lines in Absorption}


✔ → 85986:\section{\label{app:CO abs} CO Lines in Absorption}
  ↳ 87805:end


E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
F. Walter  ->  F. Walter  |  ['F. Walter']


Found 59 bibliographic references in tmp_2504.02538/ms_arxiv.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.02071-b31b1b.svg)](https://arxiv.org/abs/2504.02071) | **Stellar tidal streams around nearby spiral galaxies with deep imaging from amateur telescopes**  |
|| D. Martinez-Delgado, et al. -- incl., <mark>M. Schirmer</mark> |
|*Appeared on*| *2025-04-04*|
|*Comments*| *21 pages, 16 figures, 5 tables. Full resolution version available in this link: this https URL*|
|**Abstract**|            Tidal interactions between massive galaxies and their satellites are fundamental processes in a Universe with L-Cold Dark Matter cosmology, redistributing material into faint features that preserve records of past galactic interactions. While stellar streams in the Local Group impressively demonstrate satellite disruption, they do not constitute a statistically significant sample. Constructing a substantial catalog of stellar streams beyond the Local Group remains challenging due to the difficulties in obtaining deep, wide-field images of galaxies. Despite their potential to illuminate dark matter distribution and galaxy formation processes, stellar streams remain underutilized as cosmological probes. The Stellar Tidal Stream Survey (STSS) addresses this observational gap by leveraging amateur telescopes to obtain deep, scientific-grade images of galactic outskirts, capable of building a more statistically meaningful sample of stellar streams. Over the last decade, the STSS has acquired deep (up to surface brightness limit 28.3 mag/arcsec^2 in the r-band), wide-field images of 15 nearby Milky Way analog galaxies using a network of robotic amateur telescopes, avoiding the issues associated with "mosaicing" smaller images taken with a professional telescope. Our survey has revealed a diverse range of previously unreported faint features related to dwarf satellite accretion - including stellar streams, shells, and umbrella-like structures. We discover an ultra-diffuse galaxy (NGC150-UDG1), which shows hints of tidal tails. The STSS demonstrates the suitability of modern amateur telescopes to detect and study faint, diffuse structures in large fields around nearby spiral galaxies. Their economic and accessibility advantages enable larger samples with deep imaging, essential for testing galaxy formation models and constraining the properties of minor merger events in the local Universe.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.02136-b31b1b.svg)](https://arxiv.org/abs/2504.02136) | **Mid-infrared extinction curve for protostellar envelopes from JWST-detected embedded jet emission: the case of TMC1A**  |
|| K. D. Assani, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2025-04-04*|
|*Comments*| *Submitted to A&A*|
|**Abstract**|            Context: Dust grains are key components of the interstellar medium and play a central role in star formation, acting as catalysts for chemical reactions and as building blocks of planets. Extinction curves are essential for characterizing dust properties, but mid-infrared (MIR) extinction remains poorly constrained in protostellar environments. Gas-phase line ratios from embedded protostellar jets provide a spatially resolved method for probing extinction through protostellar envelopes, complementing background starlight approaches. Aims: We aim to derive MIR extinction curves along sightlines toward a protostellar jet embedded in an envelope and assess whether they differ from those in dense molecular clouds. Methods: We analyze JWST NIRSpec IFU and MIRI MRS observations of four positions along the blue-shifted TMC1A jet. We extract observed [Fe II] line intensities and model intrinsic ratios using the Cloudy spectral synthesis code across a range of electron densities and temperatures. By comparing observed near-IR (NIR) and MIR line ratios to Cloudy predictions, we infer the relative extinction between NIR and MIR wavelengths. Results: Electron densities (ne) derived from NIR [Fe II] lines range from ~5 x 10^4 to ~5 x 10^3 cm^-3 at scales <~350 AU. MIR extinction values show stronger reddening than the empirical dark cloud curve from McClure (2009) at similar ne and temperatures (~10^3 to 10^4 K). If MIR emission arises from cooler, lower-density gas, extinction curves remain consistent with background starlight measurements. Conclusions: This method enables spatially resolved MIR extinction curves in embedded protostellar systems. Results suggest either a change in dust size distribution (e.g., from grain growth) or that MIR emission originates from cooler, less dense regions than NIR emission. (Abstract shortened for arXiv. See PDF for full version.)         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.02338-b31b1b.svg)](https://arxiv.org/abs/2504.02338) | **Coronal and chromospheric activity of Teegarden's star**  |
|| B. Fuhrmeister, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-04-04*|
|*Comments*| *15 pages, 20 figures*|
|**Abstract**|            Teegarden's star is a late-type M-dwarf planet host, typically showing only rather low levels of activity. In this paper we present an extensive characterisation of this activity at photospheric, chromospheric, and coronal levels. We specifically investigated TESS observations of Teegarden's star, which showed two very large flares with an estimated flare fluence between 10$^{29}$ and 10$^{32}$\,erg comparable to the largest solar flares. We furthermore analysed nearly 300 CARMENES spectra and 11 ESPRESSO spectra covering all the usually used chromospheric lines in the optical from the \ion{Ca}{ii} H \& K lines at 3930\,Å\, to the \ion{He}{i} infrared triplet at 10830\,Å. These lines show different behaviour: The \ion{He}{i} infrared triplet is the only one absent in all spectra, some lines show up only during flares, and others are always present and highly variable. Specifically, the H$\alpha$ line is more or less filled in during quiescence; however, the higher Balmer lines are still observed in emission. Many chromospheric lines show a correlation with H$\alpha$ variability, which, in addition to stochastic behaviour, also shows systematic behaviour on different timescales including the rotation period. Moreover, we found several flares and also report hints of an erupting prominence, which may have led to a coronal mass ejection. Finally, we present X-ray observations of Teegarden's star (i.e. a discovery pointing obtained with the \emph{Chandra} observatory) and an extensive study with the \emph{XMM-Newton} observatory; when these two large flares were observed, one of them showed clear signatures of the Neupert effect, suggesting the production of hard X-rays in the system.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.02525-b31b1b.svg)](https://arxiv.org/abs/2504.02525) | **Detection of cosmological dipoles aligned with transverse peculiar velocities**  |
|| Y.-C. Cai, J. A. Peacock, <mark>A. d. Graaff</mark>, S. Alam |
|*Appeared on*| *2025-04-04*|
|*Comments*| *26 pages, 18 figures*|
|**Abstract**|            We present the first observations of a novel dipole signature imprinted on the CMB by transverse velocities. Cosmological peculiar velocities point towards gravitational wells and away from potential hills, reflecting a large-scale dipole in the gravitational potential, coherent over hundreds of Mpc. We predict large-scale dipoles in all fields correlated with the potential, observable via effects of gravitational lensing and the integrated Sachs-Wolfe (ISW). The ISW dipole is distinct from the small-scale moving lens effect, which has a dipole of the opposite sign. We provide a unified framework for analysing these dipoles, and make the first detections in galaxy density, CMB lensing convergence and the ISW effect. We show that the observed signals are consistent with LCDM predictions, and set limits on modified gravity. The CMB dipole signal is independent of galaxy bias, and orthogonal to the monopole correlation function, so this new observable provides additional cosmological information (abridged).         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.02294-b31b1b.svg)](https://arxiv.org/abs/2504.02294) | **Search for Fast Radio Bursts and radio pulsars from pulsing Ultraluminous X-ray Sources**  |
|| J. Bai, et al. -- incl., <mark>N. Wang</mark> |
|*Appeared on*| *2025-04-04*|
|*Comments*| *9 pages, 2 figures, 2 tables, Accepted for publication in ApJ*|
|**Abstract**|            We conducted targeted fast radio burst (FRB) and pulsar searches on eight pulsing ultraluminous X-ray sources (PULXs) using the Five-hundred-meter Aperture Spherical Radio Telescope (FAST) and the Parkes 64-meter Radio Telescope (Murriyang) to investigate whether PULXs could be progenitors of FRBs. FAST carried out 12 observations of four PULXs, totaling 8 hours, while Parkes conducted 12 observations of the remaining four PULXs, totaling 11 hours. No significant signals were detected through single-pulse and periodic searches, covering a dispersion measure (DM) range of 0-5000 pc cm$^{-3}$, placing stringent upper limits on the radio flux density from these sources. The results imply that accretion processes and dense stellar winds in PULXs likely suppress or attenuate potential coherent emission in radio band. Additionally, the beaming factor and luminosity of FRBs associated with PULXs, as well as the highly relativistic and magnetized nature of their outflows, may limit detectability. Non-detection yielded from the observations covering the full orbital phases of PULXs can also constrain the theoretical models that link FRB emission to highly magnetized neutron stars in binary systems.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.02415-b31b1b.svg)](https://arxiv.org/abs/2504.02415) | **Unveiling Spatiotemporal Properties of the Quasi-periodic Pulsations in the Balmer Continuum at 3600 Å in an X-class Solar White-light Flare**  |
|| D.-C. Song, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-04-04*|
|*Comments*| *19 pages, 6 figures, and two tables*|
|**Abstract**|            Quasi-periodic pulsations (QPPs) in the Balmer continuum of solar white-light flares (WLFs) are rarely reported, and accurately pinpointing the spatial source of flaring QPPs remains a significant challenge. We present spatiotemporal characteristics of QPPs of an X2.8 two-ribbon solar WLF (SOL2023-12-14T17:02), which was well observed by the White-light Solar Telescope (WST) aboard the Advanced Space-based Solar Observatory, with high-cadence imaging (1--2 s) in the Balmer continuum at 3600 Å. Combined with additional multi-instrument data, we find that the enhancement of the WLF in both Balmer and Paschen continua shows strong spatiotemporal correlation with hard X-ray (HXR) emissions. Notably, the pulses in the WST Balmer continuum exhibited a near-zero time lag with most HXR pulses, whereas soft X-ray and extreme ultraviolet emissions showed a lag of 2--3 s. Interestingly, quasi-harmonic QPPs with periods of $\sim$11 s and $\sim$20 s were observed in multiple wavelengths in the rising phase of the white-light continuum. Furthermore, we employed Fourier transform to spatially locate the QPPs around 11 and 20 s, revealing that they primarily originated from the east flare ribbon, which exhibited the most substantial continuum enhancement. More interestingly, we find that the west ribbon contributed significantly to the 11-second QPP but had a weaker contribution to the 20-second QPP. Moreover, the occurrence of quasi-harmonic QPPs is temporally coincident with the rapid elongation and separation motions of flare ribbons. Possible mechanisms for the quasi-harmonic QPPs have been discussed. These observations provide valuable insights into QPP modeling for solar and stellar flares.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.02035-b31b1b.svg)](https://arxiv.org/abs/2504.02035) | **Strikingly high fraction of fast rotators in Magellanic Cloud star clusters**  |
|| G. Ettorre, et al. -- incl., <mark>M. Fouesneau</mark> |
|*Appeared on*| *2025-04-04*|
|*Comments*| *Accepted for publication in MNRAS. 19 pages, 16 figures, 3 tables (additional 5 pages and 5 figures in the appendix)*|
|**Abstract**|            There has been growing evidence that the rich star clusters in the Magellanic Clouds contain significant fractions of rapidly rotating stars. In this work, we aim to constrain these fractions by studying the colour-magnitude diagrams of four star clusters, selected among those with the most striking signatures of fast rotators. Using isochrones derived from PARSEC v2.0 stellar tracks, we generate distinct stellar populations, each covering a limited interval of initial rotation rates $\omega_\mathrm{i}$, referred to as 'Partial Models' (PMs). Using optimization algorithms and Monte Carlo Markov Chains, PMs are combined to create the final best-fitting model. In our analysis, we adopt two key assumptions: a uniform age and an isotropic distribution of stellar spin axes within each cluster. The solutions are allowed to explore the entire range of $\omega_\mathrm{i}$, and different values of age, metallicity, distance and foreground extinction. We find that the rotational velocity distributions in all four clusters reveal a high fraction of stars with $\omega_\mathrm{i}$ close to the break-up value, in all cases. Specifically, the fraction of stars with $\omega_\mathrm{i}>0.7$ exceeds $80\%$ in the clusters NGC 419 of the Small Magellanic Cloud (SMC) and NGC 1831 and NGC 1866 of the Large Magellanic Cloud (LMC). For NGC 2203 of the LMC, this fraction is smaller, although it still exceeds $50\%$, confirming that also this cluster is mainly populated by fast-rotating stars.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.02305-b31b1b.svg)](https://arxiv.org/abs/2504.02305) | **Nuclear Winds Drive Large-Scale Cold Gas Outflows in Quasars during the Reionization Epoch**  |
|| Y. Zhu, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2025-04-04*|
|*Comments*| *14 pages; submitted*|
|**Abstract**|            Accreting supermassive black holes (SMBHs) regulate the evolution of their host galaxies through powerful outflows and multi-phase feedback. This process plays a crucial role in shaping SMBH-galaxy co-evolution across cosmic time, but direct evidence linking nuclear winds to large-scale cold gas outflows, particularly in high-redshift quasars, has remained elusive. Here we present statistical evidence of a connection between nuclear winds and large-scale cold gas outflows in quasars at $z \sim 5.5$. Using stacked [C II] 158 $\mu$m emission profiles from ALMA observations, which trace galactic-scale neutral gas, we compare broad absorption line (BAL) quasars -- tracing parsec- to sub-kiloparsec-scale nuclear winds -- with non-BAL quasars. The BAL stack reveals a significant (S/N=4.45) broad component in the [C II] emission, indicating high-velocity neutral gas outflows with a velocity offset of $\Delta v_{\rm b} = -2.1 \times 10^2 \, \rm km\,s^{-1}$ and a full width at half maximum of $1.18 \times 10^3 \, \rm km\,s^{-1}$, while the non-BAL stack shows no such feature. We estimate that a few percent up to one-quarter of the BAL wind energy is transferred to neutral gas on kiloparsec scales. These findings provide direct observational evidence that nuclear winds couple with galactic-scale neutral gas flows, supporting multi-phase AGN feedback models. This mechanism may contribute to explaining the diversity of $M_{\rm BH}/M_*$ ratios observed in some luminous AGN recently observed by JWST, compared to the Magorrian relation.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.02394-b31b1b.svg)](https://arxiv.org/abs/2504.02394) | **Mass, Luminosity, and Stellar Age of Early-type Stars from the LAMOST Survey**  |
|| Q. Li, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-04-04*|
|*Comments*| **|
|**Abstract**|            Mass ($M$) and luminosity ($L$) are fundamental parameters of stars but can only be measured indirectly. Typically, effective temperature ($T_{\rm eff}$), surface gravity (${\rm log}\ g$) and metallicity ([M/H]) are derived from stellar spectra, then $M$, $L$ and stellar age ($t$) can be obtained by interpolating in the grid of stellar evolutionary models. In this paper, we use the Random Forest (RF) in combination with the evolutionary grid from PARSEC 1.2S to determine $M$, $L$, $t$ and initial mass ($M_{\rm i}$) for early-type main-sequence stars ($T_{\rm eff}\geq 7,000\ {\rm K}$) as identified from the LAMOST survey. The convex hull algorithm is employed to select the main-sequence stars. The results demonstrate that the prediction precision is $32\%$ and $9\%$ for $L$ and $M$, respectively, which is comparable to that achieved by fitting evolutionary tracks. Furthermore, the predicted $L$ also aligns with Gaia's observations, with a relative difference of $36\%$. The prediction for $t$ is relatively less accurate, indicating a difference of 0.44 Gyr for two components in wide binaries. This discrepancy is due to the inconsistent metallicity measurements. For the two sets of atmospheric parameters we used, the relative differences in $L$, $M$, $t$ and $M_{\rm i}$ are $29\%$, $7\%$, $36\%$ and $7\%$, respectively. The influence of metallicity on these parameters is analyzed, with the conclusion that metallicity has the greatest impact on $t$. Consequently, two catalogs are presented, which would be useful for studying stellar populations such as the luminosity function and initial mass function of early-type stars.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.02071/./Figure6_Sep30.jpg', 'tmp_2504.02071/./Figure7_Sept30.jpg', '', '', '', '']
copying  tmp_2504.02071/./Figure6_Sep30.jpg to _build/html/
copying  tmp_2504.02071/./Figure7_Sept30.jpg to _build/html/
file not found 
file not found 
file not found 
file not found 
exported in  _build/html/2504.02071.md
    + _build/html/tmp_2504.02071/./Figure6_Sep30.jpg
    + _build/html/tmp_2504.02071/./Figure7_Sept30.jpg
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
found figures ['tmp_2504.02136/./Figures_2/iron_detections_maps_with_summed_regions_subset_0_to_None.png', 'tmp_2504.02136/./Figures_2/iron_detections_with_summed_regions_shaded_lines_subset_0_to_None.png', 'tmp_2504.02136/./Figures_2/r_obs_v_rmodel_ne_limits_x_1.2_to_1.86_caseA_2.png', 'tmp_2504.02136/./Figures_2/r_obs_v_rmodel_ne_limits_x_4_to_27_caseA_2.png', 'tmp_2504.02136/./Figures_2/delta_Alambdas_dual_limits_x_1.2_to_1.86_caseA_2.png', 'tmp_2504.02136/./Figures_2/delta_Ala

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\commentMS}[1]{\textcolor{orange}{[MS: #1]}}$
$\newcommand$</div>



<div id="title">

# Stellar tidal streams around nearby spiral galaxies with deep imaging from amateur telescopes

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.02071-b31b1b.svg)](https://arxiv.org/abs/2504.02071)<mark>Appeared on: 2025-04-04</mark> -  _21 pages, 16 figures, 5 tables. Full resolution version available in this link: this https URL_

</div>
<div id="authors">

D. Martinez-Delgado, et al. -- incl., <mark>M. Schirmer</mark>

</div>
<div id="abstract">

**Abstract:** Tidal interactions between massive galaxies and their satellites are fundamental processes in a Universe with $\Lambda$ -Cold Dark Matter ( $\Lambda$ CDM) cosmology, redistributing material into faint features that preserve records of past galactic interactions. While stellar streams in the Local Group impressively demonstrate satellite disruption, they do not constitute a statistically significant sample. Constructing a substantial catalog of stellar streams beyond the Local Group remains challenging due to the difficulties in obtaining sufficiently deep, wide-field images of galaxies. Despite their potential to illuminate dark matter distribution and galaxy formation processes overall, stellar streams remain underutilized as cosmological probes. The Stellar Tidal Stream Survey (STSS) addresses this observational gap by leveraging amateur telescopes to obtain deep, scientific-grade images of galactic outskirts, capable of building a more statistically meaningful sample of stellar streams. Over the last decade, the STSS has acquired deep (up to surface brightness limit $\sim$ 28.3 mag/arcsec $^2$ in the _r_ -band) wide-field images of 15 nearby Milky Way analog galaxies using a coordinated network of robotic amateur telescopes, avoiding the issues associated with `mosaicing’ smaller images taken with a single, professional telescope. Our survey has revealed a diverse range of previously unreported faint features related to dwarf satellite accretion— including stellar streams, shells, and umbrella-like structures. We serendipitously discover an ultra-diffuse galaxy (NGC150-UDG1) which shows hints of tidal tails. The STSS demonstrates the suitability of modern amateur telescopes to detect and study faint, diffuse structures in large fields around nearby spiral galaxies. Their economic and accessibility advantages enable larger statistical samples with deep imaging, essential for testing galaxy formation models and constraining the frequency and properties of minor merger events in the local Universe.

</div>

<div id="div_fig1">

<img src="tmp_2504.02071/./Figure6_Sep30.jpg" alt="Fig20" width="100%"/>

**Figure 20. -** Our STSS images of ESO 545-5, NGC 95, NGC 150, NGC 925, NGC 1511 and NGC 2775. North is up and East is left. See text for discussion on all six images. (*fig:cmd1*)

</div>
<div id="div_fig2">

<img src="tmp_2504.02071/./Figure7_Sept30.jpg" alt="Fig23" width="100%"/>

**Figure 23. -** Our STSS images of NGC 3041, NGC 3614, NGC3631, NGC 3631, NGC 4390, NGC 4414 and NGC 4684. See text for discussion on all six images. (*fig:cmd2*)

</div>
<div id="div_fig3">

<img src="" alt="Fig5.1" width="25%"/><img src="" alt="Fig5.2" width="25%"/><img src="" alt="Fig5.3" width="25%"/><img src="" alt="Fig5.4" width="25%"/>

**Figure 5. -** Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Nonlinear Model ResultsNonlinear Model ResultsSpectral types and photometry for stars in the
  region.Spectral types and photometry for stars in the
  region.List of nearby SNe used in this work.Summary for ISOCAM sources with mid-IR excess
(YSO candidates).Summary for ISOCAM sources with mid-IR excess
(YSO candidates). Sample stars with absolute magnitudecontinued. Sample stars with absolute magnitudecontinued.Shown in greyscale is a...Plotted above...Complexes characterisation.Line data and abundances ...Continued. (*FigVibStab*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.02071"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Mid-infrared extinction curve for protostellar envelopes from JWST-detected embedded jet emission: the case of TMC1A

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.02136-b31b1b.svg)](https://arxiv.org/abs/2504.02136)<mark>Appeared on: 2025-04-04</mark> -  _Submitted to A&A_

</div>
<div id="authors">

K. D. Assani, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** Dust grains are fundamental components of the interstellar medium and play a crucial role in star formation, serving as catalysts for chemical reactions and the building blocks of planets. Extinction curves serve as a tool for characterizing dust properties, yet mid-infrared (MIR) extinction remains less constrained in protostellar environments. Gas-phase line ratios from embedded protostellar jets offer a spatially resolved method to measure extinction from protostellar envelopes, complementing traditional background starlight techniques. We aim to derive mid-IR extinction curves along the lines of sight toward a protostellar jet embedded within an envelope and assess whether they differ from those inferred in dense molecular clouds. We analyze JWST NIRSpec IFU and MIRI MRS observations, focusing on four locations along the blue-shifted TMC1A jet. After extracting observed [ $\ion{Fe}{II}$ ] line intensities, we model intrinsic line ratios using the Cloudy spectral synthesis code across a range of electron densities and temperatures. By comparing observed near-IR (NIR) and MIR line ratios to Cloudy-predicted intrinsic ratios, we infer the relative extinction between NIR and MIR wavelengths. Electron densities ( $n_e$ ) derived from NIR [ $\ion{Fe}{II}$ ] lines range from $\sim5 \times 10^4$ to $\sim5 \times 10^3$ cm $^{-3}$ along the jet axis at scales $\lesssim$ 350 AU, serving as reference points for comparing relative NIR and MIR extinction. The derived MIR extinction values exhibit higher reddening than the empirical dark cloud curve from [ and McClure (2009)]() at the corresponding $n_e$ and temperatures ranging from a few 10 $^3$ K to $\sim$ 10 $^4$ K, as adopted from shock models. While both electron density and temperature influence the NIR-to-MIR [ $\ion{Fe}{II}$ ] line ratios, the ratios are more strongly dependent on $n_e$ over the adopted range. If the MIR emission originates from gas that is less dense and cooler than the NIR-emitting region, the inferred extinction curves remain consistent with background star-derived values. This study introduces a new line-based method for deriving spatially resolved MIR extinction curves towards embedded protostellar sources exhibiting a bright [ $\ion{Fe}{II}$ ] jet. The results suggest that protostellar envelopes may contain dust with a modified grain size distribution---such as an increased fraction of larger grains, potentially due to grain growth---if the MIR and NIR lines originate from similar regions along the same sightlines. Alternatively, if the grain size distribution has not changed (i.e., no grain growth), the MIR lines may trace cooler, less dense gas than the NIR lines along the same sightlines. This method provides a novel approach for studying dust properties in star-forming regions and can be extended to other protostellar systems to refine extinction models in embedded environments.

</div>

<div id="div_fig1">

<img src="tmp_2504.02136/./Figures_2/iron_detections_maps_with_summed_regions_subset_0_to_None.png" alt="Fig5.1" width="50%"/><img src="tmp_2504.02136/./Figures_2/iron_detections_with_summed_regions_shaded_lines_subset_0_to_None.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** [$\ion${Fe}{II}] emission lines used in this study and detected with NIRSpec and MIRI, arranged in order of increasing upper energy ($E_U/k$). **Top panels**: moment-0 maps (in units of erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$), constructed by integrating the emission across the frequency range indicated by the shaded regions in the corresponding spectra below. Although integration is performed in frequency space, the x-axis is displayed in velocity (km s$^{-1}$) for clarity. A 3-sigma threshold is applied. The maps are overlaid with circular, color-coded apertures (0.7") used consistently throughout this study. **Bottom panels**: spectra (in units of erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$ Hz$^{-1}$) extracted from each aperture, with shaded regions highlighting the velocity range used for integration, primarily focusing on blue-shifted emission. Each subplot includes the rest wavelength ($\lambda_0$), transition and upper energies ($\Delta E$, $E_U$), and Einstein-A value for reference. (*fig:iron_detections_subset*)

</div>
<div id="div_fig2">

<img src="tmp_2504.02136/./Figures_2/r_obs_v_rmodel_ne_limits_x_1.2_to_1.86_caseA_2.png" alt="Fig6.1" width="16%"/><img src="tmp_2504.02136/./Figures_2/r_obs_v_rmodel_ne_limits_x_4_to_27_caseA_2.png" alt="Fig6.2" width="16%"/><img src="tmp_2504.02136/./Figures_2/delta_Alambdas_dual_limits_x_1.2_to_1.86_caseA_2.png" alt="Fig6.3" width="16%"/><img src="tmp_2504.02136/./Figures_2/delta_Alambdas_dual_limits_x_4_to_27_caseA_2.png" alt="Fig6.4" width="16%"/><img src="tmp_2504.02136/./Figures_2/beta_1644_ne_dual_limits_x_1.2_to_1.86_caseA_2.png" alt="Fig6.5" width="16%"/><img src="tmp_2504.02136/./Figures_2/beta_1644_ne_dual_limits_x_4_to_27_caseA_2.png" alt="Fig6.6" width="16%"/>

**Figure 6. -** _NIR and MIR extinction toward the TMC1A jet_. Same as Figure 2, but extended to include additional a$^4$D lines in the NIR (left) used for $n_{\rm e}$ derivation, along with MIR lines (right). Model scatter points correspond to the derived electron density at 10,000 K (see Sect. \ref{sec:4_electron_density_NIR}), with uncertainties reflecting the uncertainty range in the determined $n_{\rm e}$ value and temperatures from 5,000–10,000 K. Commonly used NIR/MIR extinction curves are overplotted for comparison (see text). (*fig:MIR_derived_extinction_same_cond*)

</div>
<div id="div_fig3">

<img src="tmp_2504.02136/./Figures_2/r_obs_v_rmodel_ne_limits_x_1.2_to_1.86_a4dj72.png" alt="Fig1.1" width="33%"/><img src="tmp_2504.02136/./Figures_2/delta_Alambdas_dual_limits_x_1.2_to_1.86_a4dj72.png" alt="Fig1.2" width="33%"/><img src="tmp_2504.02136/./Figures_2/beta_1644_ne_dual_limits_x_1.2_to_1.86_a4dj72.png" alt="Fig1.3" width="33%"/>

**Figure 1. -** Near-infrared (NIR) extinction of a$^4$D J=7/2 lines relative to the 1.644 $\mu$m line (horizontal and vertical dashed lines.).
**Top:** Observed (\texttt{'o'}) and modeled (\texttt{'*'}) line ratios for all four jet locations in this study.
**Middle:** The difference in extinction relative to the 1.644 $\mu$m line.
**Bottom:** Derived $\beta_{\lambda}$ values, normalized by the extinction at 1.644 $\mu$m, which is computed as an average over existing NIR extinction curves. This is compared with empirically determined NIR extinction curves---C89 cardelli1989relationship, and modeled extinction curves---WD1 [ and Weingartner (2001)](), KP5 [Pontoppidan, et. al (2024)]().
We include $R_V=3.1$ and $R_V=5.5$ for applicable curves to illustrate that NIR extinction in the 1.2–1.8 $\mu$m range is largely independent of $R_V$ and shows minimal differences between independent studies. Dashed lines mark the reference line ratio, extinction difference, and $\beta_{\lambda}$ value at 1.644 $\mu$m (horizontal) and the wavelength of 1.644 $\mu$m (vertical) in all three panels. (*fig:a4dj72_extinction*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.02136"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\gaia}{{\it Gaia}\xspace}$
$\newcommand{\chandra}{{\it Chandra}\xspace}$</div>



<div id="title">

# Coronal and chromospheric activity of Teegarden's star 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.02338-b31b1b.svg)](https://arxiv.org/abs/2504.02338)<mark>Appeared on: 2025-04-04</mark> -  _15 pages, 20 figures_

</div>
<div id="authors">

B. Fuhrmeister, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Teegarden's star is a late-type M-dwarf planet host, typically showing only rather low levels of activity. In this paper we present an extensive characterisation of this activity at photospheric, chromospheric, and coronal levels.  We specifically investigated TESS observations of Teegarden's star, which showed two very large flares with an estimated flare fluence between 10 $^{29}$ and 10 $^{32}$ erg comparable to the largest solar flares.  We furthermore analysed nearly 300 CARMENES spectra and 11 ESPRESSO spectra covering all the usually used chromospheric lines in the optical from the $\ion{Ca}{ii}$ H \& K lines at 3930 Å  to the $\ion{He}{i}$ infrared triplet at 10830 Å. These lines show different behaviour: The $\ion{He}{i}$ infrared triplet is the only one absent in all spectra,  some lines show up only during flares, and  others are always present and highly variable. Specifically, the H $\alpha$ line is more or less filled in during quiescence; however, the higher Balmer lines are still observed in emission. Many chromospheric lines show a correlation with H $\alpha$ variability, which, in addition to stochastic behaviour, also shows systematic behaviour on different timescales including the rotation period.    Moreover, we found several flares and also report hints of an erupting prominence, which may have led to a coronal mass ejection. Finally, we present X-ray observations of Teegarden's star (i.e. a discovery pointing obtained with the $*Chandra*$ observatory)    and an extensive study with the $*XMM-Newton*$ observatory; when  these    two large flares were observed, one of them showed clear signatures of the Neupert effect, suggesting the production of hard X-rays in the system.

</div>

<div id="div_fig1">

<img src="tmp_2504.02338/./Fig/LindLbol.png" alt="Fig18" width="100%"/>

**Figure 18. -**  Time series of $L_{\mathrm{indicator}}/L_{\mathrm{bol}}$. For the CARMENES
  $L_{\mathrm{H}\alpha}/L_{\mathrm{bol}}$ we mark low activity states as green and blue dots, high activity states as  red dots, flares as magenta dots, and spectra with an asymmetric H$\alpha$ shape as cyan dots (see Sect. \ref{sec:halpha} for a detailed discussion). The cyan and magenta dots are labelled with the flare number also used in Fig.\ref{subtracspec}. Flare no. 14 is not shown here since the spectrum gets into absorption, and therefore no $L_{\mathrm{indicator}}/L_{\mathrm{bol}}$ can be calculated with the $\chi$ method used here since it is only defined for emission lines.
  The ESPRESSO H$\alpha$ measurements are marked as black triangles.
  The $L_{\rm X}/L_{\mathrm{bol}}$ measurement of the *Chandra* observation is marked as a black star; that of the *XMM-Newton* observation is marked as a black diamond.  The time spans of the TESS observations are marked as   small black dots connected by a black line. Since TESS is not photometrically calibrated, the position on the y-axis is arbitrary (we note here that for a blackbody of the temperature of Teegarden's star about 20 percent of the radiation is in the TESS band). The red triangles mark the positions of the clusters of the higher activity states.
 (*pEWhalpha*)

</div>
<div id="div_fig2">

<img src="tmp_2504.02338/./Fig/teegarden_xmm_om.jpg" alt="Fig1" width="100%"/>

**Figure 1. -** _ XMM-Newton_ light curve for Teegarden's star obtained with the
OM in the U band (red data points, time resolution 10 s) and
the EPIC pn detector (blue data points, time resolution 300 s). Flare III occurs at $\sim$18.6 h and Flare IV occurs at $\sim$20.9 h.
 (*fig:xmm_pn_om*)

</div>
<div id="div_fig3">

<img src="tmp_2504.02338/./Fig/flare2.jpg" alt="Fig3" width="100%"/>

**Figure 3. -**  Close-up view of _XMM-Newton_ data for Flare IV. The OM U band rate (1 sec bins) is shown as blue data points, while the arrival times of the recorded EPIC events are shown as red dots.  (*flare_2_higres*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.02338"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\be}{\begin{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\bes}{\begin{equation*}}$
$\newcommand{\ees}{\end{equation*}}$
$\newcommand{\bea}{\begin{eqnarray}}$
$\newcommand{\eea}{\end{eqnarray}}$
$\newcommand{\beas}{\begin{eqnarray*}}$
$\newcommand{\eeas}{\end{eqnarray*}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\ten}{10^{10}}$
$\newcommand{\eleven}{10^{11}}$
$\newcommand{\twelve}{10^{12}}$
$\newcommand{\thirteen}{10^{13}}$
$\newcommand{\fourteen}{10^{14}}$
$\newcommand{\Mpc}{ h^{-1}{\rm Mpc}}$
$\newcommand{\de}{{\rm d}}$
$\newcommand{\rhoin}{\rho_{\rm in}}$
$\newcommand{\rhoout}{\rho_{\rm out}}$
$\newcommand{\phiin}{\phi_{\rm in}}$
$\newcommand{\phiout}{\phi_{\rm out}}$
$\newcommand{\roll}{R_{\rm roll}}$
$\newcommand{\lamin}{\lambda_{\rm in}}$
$\newcommand{\lamout}{\lambda_{\rm out}}$
$\newcommand{\mpl}{M_{\rm Pl}}$
$\newcommand{\senv}{S_{\rm env}}$
$\newcommand{\renv}{R_{\rm env}}$
$\newcommand{\denv}{\delta_{\rm env}}$
$\newcommand{\dsc}{\delta_{\rm sc}}$
$\newcommand{\dv}{\delta_{\rm v}}$
$\newcommand{\tcr}{\textcolor{red}}$
$\newcommand{\tcb}{\textcolor{red}}$
$\newcommand\bnabla{{\boldsymbol{\nabla}}}$
$\newcommand\br{{\bf r}}$
$\newcommand\bg{{\bf g}}$
$\newcommand\bv{{\bf v}}$
$\newcommand\bx{{\bf x}}$
$\newcommand\bxhat{{\hat \bx}}$
$\newcommand{\[}{\begin{equation}}$
$\newcommand{\]}{\end{equation}}$
$\newcommand{\kms}{{ \rm km s^{-1}}}$
$\newcommand{\kmsmpc}{{ \rm km s^{-1}Mpc^{-1}}}$
$\newcommand{\hompc}{{ h \rm Mpc^{-1}}}$
$\newcommand{\mpcoh}{{ h^{-1} \rm Mpc}}$
$\newcommand\JAP{#1}{{\color{red}#1}}$
$\newcommand\SA{#1}{{\color{pink}#1}}$
$\newcommand\v{#1}{{\bf #1}}$
$\newcommand{\vec}[1]{{\boldsymbol{#1}}}$
$\newcommand{\YC}[1]{{\color{red}{#1}}}$
$\newcommand{\JP}[1]{{\color{green}{#1}}}$
$\newcommand\citejap{#1}{\citeauthor{#1} \citeyear{#1}}$
$\newcommand\JAP{#1}{{\color{red}#1}}$
$\newcommand\SA{#1}{{\color{magenta}#1}}$
$\newcommand\YC{#1}{{\color{orange}#1}}$
$\newcommand\m{@th}{\mathsurround=0pt }$
$\newcommand\eqalign{#1}{\null \vcenter{\openup1\jot \m@th$
$ \ialign{\strut\hfil\displaystyle{##}&\displaystyle{ ##}\hfil$
$ \crcr#1\crcr}} }$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\gs}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle >}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\ls}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle <}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\japsc}{\scriptscriptstyle\rm}$
$\newcommand\omit{#1 $
$}\newcommand{\fg}{f}$
$\include{aas_journals}$
$\setlength{\topmargin}{-10mm}$
$\begin{document}$
$\title[Detection of cosmological dipoles aligned with transverse peculiar velocities]{Detection of cosmological dipoles aligned with transverse peculiar velocities}$
$\author[Y.-C. Cai et al.]{$
$Yan-Chuan Cai^{1}\thanks{E-mail: cai@roe.ac.uk},$
$John A. Peacock^{1},$
$Anna de Graaff^{2},$
$Shadab Alam^{3}$
$\ ^{1}Institute for Astronomy, University of Edinburgh, Royal Observatory Edinburgh, Blackford Hill, Edinburgh EH9 3HJ, UK\ ^2Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany \ ^3Department of Theoretical Physics, Tata Institute of Fundamental Research, Homi Bhabha Road, Mumbai 400005, India$
$}$
$\maketitle$
$\begin{abstract}$
$Peculiar velocities encode rich cosmological information, but their transverse components are hard to measure. Here, we present the first observations of a novel effect of transverse velocities: the dipole signatures that they imprint on the Cosmic Microwave Background.$
$The peculiar velocity field points towards gravitational wells and away from potential hills, reflecting a large-scale dipole in the gravitational potential, coherent over hundreds of Mpc. Analogous dipoles will also exist in all other fields that correlate with the potential. These dipoles are readily observed in projection on the CMB sky via gravitational lensing and the integrated Sachs-Wolfe (ISW) effect -- both of which correlate with transverse peculiar velocities. The large-scale ISW dipole is distinct from the small-scale moving lens effect, which has a dipole of the opposite sign. We new a unified framework for analysing these velocity-related dipoles and demonstrate how stacking can extract the signal from sky maps of galaxy properties, CMB temperature, and lensing. We show that the CMB dipole signal is independent of galaxy bias, and orthogonal to the usual monopole correlation function, so this new observable news additional cosmological information. We present the first detections of the dipole signal in (i) galaxy density; (ii) CMB lensing convergence; and (iii) CMB temperature -- interpreted as the ISW effect -- using galaxies from the SDSS-III BOSS survey and CMB maps from {\it Planck}. We show that the observed signals are consistent with \LambdaCDM predictions, and use the combined lensing and ISW results to set limits on linearised models of modified gravity.$
$\end{abstract}$
$\begin{keywords}$
$gravitation -- gravitational lensing: weak -- methods: analytical -- methods: observational -- cosmic background radiation -- cosmological parameters -- large-scale structure of Universe -- cosmology: observations$
$\end{keywords}$
$\n\end{document}\end{equation}}\end{equation}}\end{eqnarray*}}\end{eqnarray}}\end{equation*}}\end{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\bes}{\begin{equation*}}$
$\newcommand{\ees}{\end{equation*}}$
$\newcommand{\bea}{\begin{eqnarray}}$
$\newcommand{\eea}{\end{eqnarray}}$
$\newcommand{\beas}{\begin{eqnarray*}}$
$\newcommand{\eeas}{\end{eqnarray*}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\ten}{10^{10}}$
$\newcommand{\eleven}{10^{11}}$
$\newcommand{\twelve}{10^{12}}$
$\newcommand{\thirteen}{10^{13}}$
$\newcommand{\fourteen}{10^{14}}$
$\newcommand{\Mpc}{ h^{-1}{\rm Mpc}}$
$\newcommand{\de}{{\rm d}}$
$\newcommand{\rhoin}{\rho_{\rm in}}$
$\newcommand{\rhoout}{\rho_{\rm out}}$
$\newcommand{\phiin}{\phi_{\rm in}}$
$\newcommand{\phiout}{\phi_{\rm out}}$
$\newcommand{\roll}{R_{\rm roll}}$
$\newcommand{\lamin}{\lambda_{\rm in}}$
$\newcommand{\lamout}{\lambda_{\rm out}}$
$\newcommand{\mpl}{M_{\rm Pl}}$
$\newcommand{\senv}{S_{\rm env}}$
$\newcommand{\renv}{R_{\rm env}}$
$\newcommand{\denv}{\delta_{\rm env}}$
$\newcommand{\dsc}{\delta_{\rm sc}}$
$\newcommand{\dv}{\delta_{\rm v}}$
$\newcommand{\tcr}{\textcolor{red}}$
$\newcommand{\tcb}{\textcolor{red}}$
$\newcommand\bnabla{{\boldsymbol{\nabla}}}$
$\newcommand\br{{\bf r}}$
$\newcommand\bg{{\bf g}}$
$\newcommand\bv{{\bf v}}$
$\newcommand\bx{{\bf x}}$
$\newcommand\bxhat{{\hat \bx}}$
$\newcommand{\[}{\begin{equation}}$
$\newcommand{\]}{\end{equation}}$
$\newcommand{\kms}{{ \rm km s^{-1}}}$
$\newcommand{\kmsmpc}{{ \rm km s^{-1}Mpc^{-1}}}$
$\newcommand{\hompc}{{ h \rm Mpc^{-1}}}$
$\newcommand{\mpcoh}{{ h^{-1} \rm Mpc}}$
$\newcommand\JAP{#1}{{\color{red}#1}}$
$\newcommand\SA{#1}{{\color{pink}#1}}$
$\newcommand\v{#1}{{\bf #1}}$
$\newcommand{\vec}[1]{{\boldsymbol{#1}}}$
$\newcommand{\YC}[1]{{\color{red}{#1}}}$
$\newcommand{\JP}[1]{{\color{green}{#1}}}$
$\newcommand\citejap{#1}{\citeauthor{#1} \citeyear{#1}}$
$\newcommand\JAP{#1}{{\color{red}#1}}$
$\newcommand\SA{#1}{{\color{magenta}#1}}$
$\newcommand\YC{#1}{{\color{orange}#1}}$
$\newcommand\m{@th}{\mathsurround=0pt }$
$\newcommand\eqalign{#1}{\null \vcenter{\openup1\jot \m@th$
$ \ialign{\strut\hfil\displaystyle{##}&\displaystyle{ ##}\hfil$
$ \crcr#1\crcr}} }$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\gs}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle >}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\ls}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle <}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\japsc}{\scriptscriptstyle\rm}$
$\newcommand\omit{#1 $
$}\newcommand{\fg}{f}$
$\include{aas_journals}$
$\setlength{\topmargin}{-10mm}$
$\begin{document}$
$\title[Detection of cosmological dipoles aligned with transverse peculiar velocities]{Detection of cosmological dipoles aligned with transverse peculiar velocities}$
$\author[Y.-C. Cai et al.]{$
$Yan-Chuan Cai^{1}\thanks{E-mail: cai@roe.ac.uk},$
$John A. Peacock^{1},$
$Anna de Graaff^{2},$
$Shadab Alam^{3}$
$\ ^{1}Institute for Astronomy, University of Edinburgh, Royal Observatory Edinburgh, Blackford Hill, Edinburgh EH9 3HJ, UK\ ^2Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany \ ^3Department of Theoretical Physics, Tata Institute of Fundamental Research, Homi Bhabha Road, Mumbai 400005, India$
$}$
$\maketitle$
$\begin{abstract}$
$Peculiar velocities encode rich cosmological information, but their transverse components are hard to measure. Here, we present the first observations of a novel effect of transverse velocities: the dipole signatures that they imprint on the Cosmic Microwave Background.$
$The peculiar velocity field points towards gravitational wells and away from potential hills, reflecting a large-scale dipole in the gravitational potential, coherent over hundreds of Mpc. Analogous dipoles will also exist in all other fields that correlate with the potential. These dipoles are readily observed in projection on the CMB sky via gravitational lensing and the integrated Sachs-Wolfe (ISW) effect -- both of which correlate with transverse peculiar velocities. The large-scale ISW dipole is distinct from the small-scale moving lens effect, which has a dipole of the opposite sign. We new a unified framework for analysing these velocity-related dipoles and demonstrate how stacking can extract the signal from sky maps of galaxy properties, CMB temperature, and lensing. We show that the CMB dipole signal is independent of galaxy bias, and orthogonal to the usual monopole correlation function, so this new observable news additional cosmological information. We present the first detections of the dipole signal in (i) galaxy density; (ii) CMB lensing convergence; and (iii) CMB temperature -- interpreted as the ISW effect -- using galaxies from the SDSS-III BOSS survey and CMB maps from {\it Planck}. We show that the observed signals are consistent with \LambdaCDM predictions, and use the combined lensing and ISW results to set limits on linearised models of modified gravity.$
$\end{abstract}$
$\begin{keywords}$
$gravitation -- gravitational lensing: weak -- methods: analytical -- methods: observational -- cosmic background radiation -- cosmological parameters -- large-scale structure of Universe -- cosmology: observations$
$\end{keywords}$
$\n\end{document}\end{equation}}\end{equation}}\end{eqnarray*}}\end{eqnarray}}\end{equation*}}$
$\newcommand{\ees}{\end{equation*}}$
$\newcommand{\bea}{\begin{eqnarray}}$
$\newcommand{\eea}{\end{eqnarray}}$
$\newcommand{\beas}{\begin{eqnarray*}}$
$\newcommand{\eeas}{\end{eqnarray*}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\ten}{10^{10}}$
$\newcommand{\eleven}{10^{11}}$
$\newcommand{\twelve}{10^{12}}$
$\newcommand{\thirteen}{10^{13}}$
$\newcommand{\fourteen}{10^{14}}$
$\newcommand{\Mpc}{ h^{-1}{\rm Mpc}}$
$\newcommand{\de}{{\rm d}}$
$\newcommand{\rhoin}{\rho_{\rm in}}$
$\newcommand{\rhoout}{\rho_{\rm out}}$
$\newcommand{\phiin}{\phi_{\rm in}}$
$\newcommand{\phiout}{\phi_{\rm out}}$
$\newcommand{\roll}{R_{\rm roll}}$
$\newcommand{\lamin}{\lambda_{\rm in}}$
$\newcommand{\lamout}{\lambda_{\rm out}}$
$\newcommand{\mpl}{M_{\rm Pl}}$
$\newcommand{\senv}{S_{\rm env}}$
$\newcommand{\renv}{R_{\rm env}}$
$\newcommand{\denv}{\delta_{\rm env}}$
$\newcommand{\dsc}{\delta_{\rm sc}}$
$\newcommand{\dv}{\delta_{\rm v}}$
$\newcommand{\tcr}{\textcolor{red}}$
$\newcommand{\tcb}{\textcolor{red}}$
$\newcommand\bnabla{{\boldsymbol{\nabla}}}$
$\newcommand\br{{\bf r}}$
$\newcommand\bg{{\bf g}}$
$\newcommand\bv{{\bf v}}$
$\newcommand\bx{{\bf x}}$
$\newcommand\bxhat{{\hat \bx}}$
$\newcommand{\[}{\begin{equation}}$
$\newcommand{\]}{\end{equation}}$
$\newcommand{\kms}{{ \rm km s^{-1}}}$
$\newcommand{\kmsmpc}{{ \rm km s^{-1}Mpc^{-1}}}$
$\newcommand{\hompc}{{ h \rm Mpc^{-1}}}$
$\newcommand{\mpcoh}{{ h^{-1} \rm Mpc}}$
$\newcommand\JAP{#1}{{\color{red}#1}}$
$\newcommand\SA{#1}{{\color{pink}#1}}$
$\newcommand\v{#1}{{\bf #1}}$
$\newcommand{\vec}[1]{{\boldsymbol{#1}}}$
$\newcommand{\YC}[1]{{\color{red}{#1}}}$
$\newcommand{\JP}[1]{{\color{green}{#1}}}$
$\newcommand\citejap{#1}{\citeauthor{#1} \citeyear{#1}}$
$\newcommand\JAP{#1}{{\color{red}#1}}$
$\newcommand\SA{#1}{{\color{magenta}#1}}$
$\newcommand\YC{#1}{{\color{orange}#1}}$
$\newcommand\m{@th}{\mathsurround=0pt }$
$\newcommand\eqalign{#1}{\null \vcenter{\openup1\jot \m@th$
$ \ialign{\strut\hfil\displaystyle{##}&\displaystyle{ ##}\hfil$
$ \crcr#1\crcr}} }$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\gs}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle >}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\ls}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle <}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\japsc}{\scriptscriptstyle\rm}$
$\newcommand\omit{#1 $
$}\newcommand{\fg}{f}$
$\include{aas_journals}$
$\setlength{\topmargin}{-10mm}$
$\begin{document}$
$\title[Detection of cosmological dipoles aligned with transverse peculiar velocities]{Detection of cosmological dipoles aligned with transverse peculiar velocities}$
$\author[Y.-C. Cai et al.]{$
$Yan-Chuan Cai^{1}\thanks{E-mail: cai@roe.ac.uk},$
$John A. Peacock^{1},$
$Anna de Graaff^{2},$
$Shadab Alam^{3}$
$\ ^{1}Institute for Astronomy, University of Edinburgh, Royal Observatory Edinburgh, Blackford Hill, Edinburgh EH9 3HJ, UK\ ^2Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany \ ^3Department of Theoretical Physics, Tata Institute of Fundamental Research, Homi Bhabha Road, Mumbai 400005, India$
$}$
$\maketitle$
$\begin{abstract}$
$Peculiar velocities encode rich cosmological information, but their transverse components are hard to measure. Here, we present the first observations of a novel effect of transverse velocities: the dipole signatures that they imprint on the Cosmic Microwave Background.$
$The peculiar velocity field points towards gravitational wells and away from potential hills, reflecting a large-scale dipole in the gravitational potential, coherent over hundreds of Mpc. Analogous dipoles will also exist in all other fields that correlate with the potential. These dipoles are readily observed in projection on the CMB sky via gravitational lensing and the integrated Sachs-Wolfe (ISW) effect -- both of which correlate with transverse peculiar velocities. The large-scale ISW dipole is distinct from the small-scale moving lens effect, which has a dipole of the opposite sign. We new a unified framework for analysing these velocity-related dipoles and demonstrate how stacking can extract the signal from sky maps of galaxy properties, CMB temperature, and lensing. We show that the CMB dipole signal is independent of galaxy bias, and orthogonal to the usual monopole correlation function, so this new observable news additional cosmological information. We present the first detections of the dipole signal in (i) galaxy density; (ii) CMB lensing convergence; and (iii) CMB temperature -- interpreted as the ISW effect -- using galaxies from the SDSS-III BOSS survey and CMB maps from {\it Planck}. We show that the observed signals are consistent with \LambdaCDM predictions, and use the combined lensing and ISW results to set limits on linearised models of modified gravity.$
$\end{abstract}$
$\begin{keywords}$
$gravitation -- gravitational lensing: weak -- methods: analytical -- methods: observational -- cosmic background radiation -- cosmological parameters -- large-scale structure of Universe -- cosmology: observations$
$\end{keywords}$
$\n\end{document}\end{equation}}\end{equation}}\end{eqnarray*}}\end{eqnarray}}$
$\newcommand{\eea}{\end{eqnarray}}$
$\newcommand{\beas}{\begin{eqnarray*}}$
$\newcommand{\eeas}{\end{eqnarray*}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\ten}{10^{10}}$
$\newcommand{\eleven}{10^{11}}$
$\newcommand{\twelve}{10^{12}}$
$\newcommand{\thirteen}{10^{13}}$
$\newcommand{\fourteen}{10^{14}}$
$\newcommand{\Mpc}{ h^{-1}{\rm Mpc}}$
$\newcommand{\de}{{\rm d}}$
$\newcommand{\rhoin}{\rho_{\rm in}}$
$\newcommand{\rhoout}{\rho_{\rm out}}$
$\newcommand{\phiin}{\phi_{\rm in}}$
$\newcommand{\phiout}{\phi_{\rm out}}$
$\newcommand{\roll}{R_{\rm roll}}$
$\newcommand{\lamin}{\lambda_{\rm in}}$
$\newcommand{\lamout}{\lambda_{\rm out}}$
$\newcommand{\mpl}{M_{\rm Pl}}$
$\newcommand{\senv}{S_{\rm env}}$
$\newcommand{\renv}{R_{\rm env}}$
$\newcommand{\denv}{\delta_{\rm env}}$
$\newcommand{\dsc}{\delta_{\rm sc}}$
$\newcommand{\dv}{\delta_{\rm v}}$
$\newcommand{\tcr}{\textcolor{red}}$
$\newcommand{\tcb}{\textcolor{red}}$
$\newcommand\bnabla{{\boldsymbol{\nabla}}}$
$\newcommand\br{{\bf r}}$
$\newcommand\bg{{\bf g}}$
$\newcommand\bv{{\bf v}}$
$\newcommand\bx{{\bf x}}$
$\newcommand\bxhat{{\hat \bx}}$
$\newcommand{\[}{\begin{equation}}$
$\newcommand{\]}{\end{equation}}$
$\newcommand{\kms}{{ \rm km s^{-1}}}$
$\newcommand{\kmsmpc}{{ \rm km s^{-1}Mpc^{-1}}}$
$\newcommand{\hompc}{{ h \rm Mpc^{-1}}}$
$\newcommand{\mpcoh}{{ h^{-1} \rm Mpc}}$
$\newcommand\JAP{#1}{{\color{red}#1}}$
$\newcommand\SA{#1}{{\color{pink}#1}}$
$\newcommand\v{#1}{{\bf #1}}$
$\newcommand{\vec}[1]{{\boldsymbol{#1}}}$
$\newcommand{\YC}[1]{{\color{red}{#1}}}$
$\newcommand{\JP}[1]{{\color{green}{#1}}}$
$\newcommand\citejap{#1}{\citeauthor{#1} \citeyear{#1}}$
$\newcommand\JAP{#1}{{\color{red}#1}}$
$\newcommand\SA{#1}{{\color{magenta}#1}}$
$\newcommand\YC{#1}{{\color{orange}#1}}$
$\newcommand\m{@th}{\mathsurround=0pt }$
$\newcommand\eqalign{#1}{\null \vcenter{\openup1\jot \m@th$
$ \ialign{\strut\hfil\displaystyle{##}&\displaystyle{ ##}\hfil$
$ \crcr#1\crcr}} }$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\gs}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle >}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\ls}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle <}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\japsc}{\scriptscriptstyle\rm}$
$\newcommand\omit{#1 $
$}\newcommand{\fg}{f}$
$\include{aas_journals}$
$\setlength{\topmargin}{-10mm}$
$\begin{document}$
$\title[Detection of cosmological dipoles aligned with transverse peculiar velocities]{Detection of cosmological dipoles aligned with transverse peculiar velocities}$
$\author[Y.-C. Cai et al.]{$
$Yan-Chuan Cai^{1}\thanks{E-mail: cai@roe.ac.uk},$
$John A. Peacock^{1},$
$Anna de Graaff^{2},$
$Shadab Alam^{3}$
$\ ^{1}Institute for Astronomy, University of Edinburgh, Royal Observatory Edinburgh, Blackford Hill, Edinburgh EH9 3HJ, UK\ ^2Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany \ ^3Department of Theoretical Physics, Tata Institute of Fundamental Research, Homi Bhabha Road, Mumbai 400005, India$
$}$
$\maketitle$
$\begin{abstract}$
$Peculiar velocities encode rich cosmological information, but their transverse components are hard to measure. Here, we present the first observations of a novel effect of transverse velocities: the dipole signatures that they imprint on the Cosmic Microwave Background.$
$The peculiar velocity field points towards gravitational wells and away from potential hills, reflecting a large-scale dipole in the gravitational potential, coherent over hundreds of Mpc. Analogous dipoles will also exist in all other fields that correlate with the potential. These dipoles are readily observed in projection on the CMB sky via gravitational lensing and the integrated Sachs-Wolfe (ISW) effect -- both of which correlate with transverse peculiar velocities. The large-scale ISW dipole is distinct from the small-scale moving lens effect, which has a dipole of the opposite sign. We new a unified framework for analysing these velocity-related dipoles and demonstrate how stacking can extract the signal from sky maps of galaxy properties, CMB temperature, and lensing. We show that the CMB dipole signal is independent of galaxy bias, and orthogonal to the usual monopole correlation function, so this new observable news additional cosmological information. We present the first detections of the dipole signal in (i) galaxy density; (ii) CMB lensing convergence; and (iii) CMB temperature -- interpreted as the ISW effect -- using galaxies from the SDSS-III BOSS survey and CMB maps from {\it Planck}. We show that the observed signals are consistent with \LambdaCDM predictions, and use the combined lensing and ISW results to set limits on linearised models of modified gravity.$
$\end{abstract}$
$\begin{keywords}$
$gravitation -- gravitational lensing: weak -- methods: analytical -- methods: observational -- cosmic background radiation -- cosmological parameters -- large-scale structure of Universe -- cosmology: observations$
$\end{keywords}$
$\n\end{document}\end{equation}}\end{equation}}\end{eqnarray*}}$
$\newcommand{\eeas}{\end{eqnarray*}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\ten}{10^{10}}$
$\newcommand{\eleven}{10^{11}}$
$\newcommand{\twelve}{10^{12}}$
$\newcommand{\thirteen}{10^{13}}$
$\newcommand{\fourteen}{10^{14}}$
$\newcommand{\Mpc}{ h^{-1}{\rm Mpc}}$
$\newcommand{\de}{{\rm d}}$
$\newcommand{\rhoin}{\rho_{\rm in}}$
$\newcommand{\rhoout}{\rho_{\rm out}}$
$\newcommand{\phiin}{\phi_{\rm in}}$
$\newcommand{\phiout}{\phi_{\rm out}}$
$\newcommand{\roll}{R_{\rm roll}}$
$\newcommand{\lamin}{\lambda_{\rm in}}$
$\newcommand{\lamout}{\lambda_{\rm out}}$
$\newcommand{\mpl}{M_{\rm Pl}}$
$\newcommand{\senv}{S_{\rm env}}$
$\newcommand{\renv}{R_{\rm env}}$
$\newcommand{\denv}{\delta_{\rm env}}$
$\newcommand{\dsc}{\delta_{\rm sc}}$
$\newcommand{\dv}{\delta_{\rm v}}$
$\newcommand{\tcr}{\textcolor{red}}$
$\newcommand{\tcb}{\textcolor{red}}$
$\newcommand{\YC}[1]{{\color{red}{#1}}}$
$\newcommand{\JP}[1]{{\color{green}{#1}}}$
$\newcommand{\vec}[1]{{\boldsymbol{#1}}}$
$\newcommand{\theenumi}{(\arabic{enumi})}$
$\newcommand\bnabla{{\boldsymbol{\nabla}}}$
$\newcommand\br{{\bf r}}$
$\newcommand\bg{{\bf g}}$
$\newcommand\bv{{\bf v}}$
$\newcommand\bx{{\bf x}}$
$\newcommand\bxhat{{\hat \bx}}$
$\newcommand{\[}{\begin{equation}}$
$\newcommand{\]}{\end{equation}}$
$\newcommand{\kms}{{ \rm km s^{-1}}}$
$\newcommand{\kmsmpc}{{ \rm km s^{-1}Mpc^{-1}}}$
$\newcommand{\hompc}{{ h \rm Mpc^{-1}}}$
$\newcommand{\mpcoh}{{ h^{-1} \rm Mpc}}$
$\newcommand\JAP{#1}{{\color{red}#1}}$
$\newcommand\SA{#1}{{\color{pink}#1}}$
$\newcommand\v{#1}{{\bf #1}}$
$\newcommand{\vec}[1]{{\boldsymbol{#1}}}$
$\newcommand{\YC}[1]{{\color{red}{#1}}}$
$\newcommand{\JP}[1]{{\color{green}{#1}}}$
$\newcommand\citejap{#1}{\citeauthor{#1} \citeyear{#1}}$
$\newcommand\JAP{#1}{{\color{red}#1}}$
$\newcommand\SA{#1}{{\color{magenta}#1}}$
$\newcommand\YC{#1}{{\color{orange}#1}}$
$\newcommand\m{@th}{\mathsurround=0pt }$
$\newcommand\eqalign{#1}{\null \vcenter{\openup1\jot \m@th$
$ \ialign{\strut\hfil\displaystyle{##}&\displaystyle{ ##}\hfil$
$ \crcr#1\crcr}} }$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\gs}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle >}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\ls}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle <}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\japsc}{\scriptscriptstyle\rm}$
$\newcommand\omit{#1 $
$}\newcommand{\fg}{f}$
$\include{aas_journals}$
$\setlength{\topmargin}{-10mm}$
$\begin{document}$
$\title[Detection of cosmological dipoles aligned with transverse peculiar velocities]{Detection of cosmological dipoles aligned with transverse peculiar velocities}$
$\author[Y.-C. Cai et al.]{$
$Yan-Chuan Cai^{1}\thanks{E-mail: cai@roe.ac.uk},$
$John A. Peacock^{1},$
$Anna de Graaff^{2},$
$Shadab Alam^{3}$
$\ ^{1}Institute for Astronomy, University of Edinburgh, Royal Observatory Edinburgh, Blackford Hill, Edinburgh EH9 3HJ, UK\ ^2Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany \ ^3Department of Theoretical Physics, Tata Institute of Fundamental Research, Homi Bhabha Road, Mumbai 400005, India$
$}$
$\maketitle$
$\begin{abstract}$
$Peculiar velocities encode rich cosmological information, but their transverse components are hard to measure. Here, we present the first observations of a novel effect of transverse velocities: the dipole signatures that they imprint on the Cosmic Microwave Background.$
$The peculiar velocity field points towards gravitational wells and away from potential hills, reflecting a large-scale dipole in the gravitational potential, coherent over hundreds of Mpc. Analogous dipoles will also exist in all other fields that correlate with the potential. These dipoles are readily observed in projection on the CMB sky via gravitational lensing and the integrated Sachs-Wolfe (ISW) effect -- both of which correlate with transverse peculiar velocities. The large-scale ISW dipole is distinct from the small-scale moving lens effect, which has a dipole of the opposite sign. We new a unified framework for analysing these velocity-related dipoles and demonstrate how stacking can extract the signal from sky maps of galaxy properties, CMB temperature, and lensing. We show that the CMB dipole signal is independent of galaxy bias, and orthogonal to the usual monopole correlation function, so this new observable news additional cosmological information. We present the first detections of the dipole signal in (i) galaxy density; (ii) CMB lensing convergence; and (iii) CMB temperature -- interpreted as the ISW effect -- using galaxies from the SDSS-III BOSS survey and CMB maps from {\it Planck}. We show that the observed signals are consistent with \LambdaCDM predictions, and use the combined lensing and ISW results to set limits on linearised models of modified gravity.$
$\end{abstract}$
$\begin{keywords}$
$gravitation -- gravitational lensing: weak -- methods: analytical -- methods: observational -- cosmic background radiation -- cosmological parameters -- large-scale structure of Universe -- cosmology: observations$
$\end{keywords}$
$\n\end{document}\end{equation}}\end{equation}}$
$\newcommand{\]}{\end{equation}}$
$\newcommand{\kms}{{ \rm km s^{-1}}}$
$\newcommand{\kmsmpc}{{ \rm km s^{-1}Mpc^{-1}}}$
$\newcommand{\hompc}{{ h \rm Mpc^{-1}}}$
$\newcommand{\mpcoh}{{ h^{-1} \rm Mpc}}$
$\newcommand\JAP{#1}$
$\newcommand\SA{#1}$
$\newcommand\v{#1}$
$\newcommand\citejap{#1}$
$\newcommand\JAP{#1}$
$\newcommand\SA{#1}$
$\newcommand\YC{#1}$
$\newcommand\m{@th}$
$\newcommand\eqalign{#1}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\gs}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle >}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\ls}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle <}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\japsc}{\scriptscriptstyle\rm}$
$\newcommand\omit{#1 $
$}\newcommand{\fg}{f}$
$\include{aas_journals}$
$\setlength{\topmargin}{-10mm}$
$\begin{document}$
$\title[Detection of cosmological dipoles aligned with transverse peculiar velocities]{Detection of cosmological dipoles aligned with transverse peculiar velocities}$
$\author[Y.-C. Cai et al.]{$
$Yan-Chuan Cai^{1}\thanks{E-mail: cai@roe.ac.uk},$
$John A. Peacock^{1},$
$Anna de Graaff^{2},$
$Shadab Alam^{3}$
$\ ^{1}Institute for Astronomy, University of Edinburgh, Royal Observatory Edinburgh, Blackford Hill, Edinburgh EH9 3HJ, UK\ ^2Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany \ ^3Department of Theoretical Physics, Tata Institute of Fundamental Research, Homi Bhabha Road, Mumbai 400005, India$
$}$
$\maketitle$
$\begin{abstract}$
$Peculiar velocities encode rich cosmological information, but their transverse components are hard to measure. Here, we present the first observations of a novel effect of transverse velocities: the dipole signatures that they imprint on the Cosmic Microwave Background.$
$The peculiar velocity field points towards gravitational wells and away from potential hills, reflecting a large-scale dipole in the gravitational potential, coherent over hundreds of Mpc. Analogous dipoles will also exist in all other fields that correlate with the potential. These dipoles are readily observed in projection on the CMB sky via gravitational lensing and the integrated Sachs-Wolfe (ISW) effect -- both of which correlate with transverse peculiar velocities. The large-scale ISW dipole is distinct from the small-scale moving lens effect, which has a dipole of the opposite sign. We new a unified framework for analysing these velocity-related dipoles and demonstrate how stacking can extract the signal from sky maps of galaxy properties, CMB temperature, and lensing. We show that the CMB dipole signal is independent of galaxy bias, and orthogonal to the usual monopole correlation function, so this new observable news additional cosmological information. We present the first detections of the dipole signal in (i) galaxy density; (ii) CMB lensing convergence; and (iii) CMB temperature -- interpreted as the ISW effect -- using galaxies from the SDSS-III BOSS survey and CMB maps from {\it Planck}. We show that the observed signals are consistent with \LambdaCDM predictions, and use the combined lensing and ISW results to set limits on linearised models of modified gravity.$
$\end{abstract}$
$\begin{keywords}$
$gravitation -- gravitational lensing: weak -- methods: analytical -- methods: observational -- cosmic background radiation -- cosmological parameters -- large-scale structure of Universe -- cosmology: observations$
$\end{keywords}$
$\n\end{document}\end{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\gs}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle >}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\ls}{\mathrel{\lower0.6ex\hbox{\buildrel{\textstyle <}$
$ \over{\scriptstyle \sim}}}}$
$\newcommand{\japsc}{\scriptscriptstyle\rm}$
$\newcommand\omit{#1 $
$}$
$\newcommand{\fg}{f}$</div>



<div id="title">

# Detection of cosmological dipoles aligned with transverse peculiar velocities

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.02525-b31b1b.svg)](https://arxiv.org/abs/2504.02525)<mark>Appeared on: 2025-04-04</mark> -  _26 pages, 18 figures_

</div>
<div id="authors">

Y.-C. Cai, J. A. Peacock, <mark>A. d. Graaff</mark>, S. Alam

</div>
<div id="abstract">

**Abstract:** Peculiar velocities encode rich cosmological information, but their transverse components are hard to measure. Here, we present the first observations of a novel effect of transverse velocities: the dipole signatures that they imprint on the Cosmic Microwave Background.The peculiar velocity field points towards gravitational wells and away from potential hills, reflecting a large-scale dipole in the gravitational potential, coherent over hundreds of Mpc. Analogous dipoles will also exist in all other fields that correlate with the potential. These dipoles are readily observed in projection on the CMB sky via gravitational lensing and the integrated Sachs-Wolfe (ISW) effect -- both of which correlate with transverse peculiar velocities. The large-scale ISW dipole is distinct from the small-scale moving lens effect, which has a dipole of the opposite sign. We provide a unified framework for analysing these velocity-related dipoles and demonstrate how stacking can extract the signal from sky maps of galaxy properties, CMB temperature, and lensing. We show that the CMB dipole signal is independent of galaxy bias, and orthogonal to the usual monopole correlation function, so this new observable provides additional cosmological information. We present the first detections of the dipole signal in (i) galaxy density; (ii) CMB lensing convergence; and (iii) CMB temperature -- interpreted as the ISW effect -- using galaxies from the SDSS-III BOSS survey and CMB maps from $_ Planck_$ . We show that the observed signals are consistent with $\Lambda$ CDM predictions, and use the combined lensing and ISW results to set limits on linearised models of modified gravity.

</div>

<div id="div_fig1">

<img src="tmp_2504.02525/./Plots/v-correlation-2D_FullBox.png" alt="Fig3.1" width="50%"/><img src="tmp_2504.02525/./Plots/v-correlation-2D_vx_vy_FullBox.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** Top panels: $z=0.55$ stacked 2D velocity field predicted from linear theory as described in Section \ref{sec:vcorrelations}(right) and the measurement from simulations, as discussed in Appendix \ref{appA}(left).
            Colours represents the amplitude of velocities, and arrows indicate velocity vectors. We can see that the velocity vectors at the centre of the plots correctly indicate that matter flows from low density to high density regions. Bottom-left and bottom-right panels show the $x$ and $y$-component of the velocity field according to linear theory, revealing a dipole and a quadrupole, respectively. (*v-correlation-2D*)

</div>
<div id="div_fig2">

<img src="tmp_2504.02525/./Plots/v-ISW-2D_400Mpc.png" alt="Fig5" width="100%"/>

**Figure 5. -** The ISW temperature fluctuations induced by the large-scale velocity field shown in Fig. \ref{v-correlation-2D}, calculated using Eq. (\ref{eq:v-ISW}). The signal is integrated over a slab of LOS depth $400$\mpcoh$$. Left: measurement from N-body simulations, as as discussed in Appendix \ref{appA}; right: linear theory. For a closer comparison with BOSS data, this figure shows results at $z=0.55$, rather than $z=0$.
     (*Tisw-2D*)

</div>
<div id="div_fig3">

<img src="tmp_2504.02525/./Plots/slab_delta_rescaled.png" alt="Fig4.1" width="50%"/><img src="tmp_2504.02525/./Plots/slab_phi_rescaled.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** The dipole amplitude according to Eq. (\ref{eq:dipole_amplitude}). This is calculated at $z=0$ according to linear theory, and scales in proportion to the linear fluctuation growth factor. The LH panel shows the case of density; the RH panel shows the case of gravitational potential, allowing for a flip in direction. Red lines show the idealised case of an infinitesimal slice, while green lines show the effect of nonzero extent along the line of sight. In the latter case, uniform slabs are considered, of thickness 50, 100, 200, $400$\mpcoh$$.
     (*fig:scalar_corr*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.02525"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\fwidth}{8.8cm}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\btdnote}[1]$
$\newcommand{\omittext}[1]$
$\newcommand{\newtext}[1]{{\color{blue}#1}}$
$\newcommand{\subtext}[2]{{\color{red}#1}{\color{blue}#2}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\topfraction}{0.99}$</div>



<div id="title">

#         $\vspace*{-2.0em}{\normalsize\rm Accepted for publication in _ The Astrophysical Journal Letters_}$\         $\vspace*{1.0em}** Detection of Deuterated Hydrocarbon Nanoparticles in the          Whirlpool Galaxy, M51**$	

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.02538-b31b1b.svg)](https://arxiv.org/abs/2504.02538)<mark>Appeared on: 2025-04-04</mark> -  _Accepted for publication in Astrophysical Journal Letters_

</div>
<div id="authors">

B.~T.~Draine, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** Deuteration of hydrocarbon material, including polycyclic aromatichydrocarbons (PAHs), has been proposed to account for the lowgas-phase abundances of D in the interstellar medium.  JWST spectra offour star-forming regions in M51 show an emission feature, withcentral wavelength $\sim4.647\micron$ and FWHM $0.0265\micron$ ,corresponding to the C--D stretching mode in aliphatic hydrocarbons.The emitting aliphatic material is estimated to have ${\rm D/H})_{\rm aliph}\approx 0.17\pm0.02$ -- a factor $\sim10^4$ enrichment relative to the overall interstellar medium (ISM).  On $\sim50 $ pc scales, deuteration levels toward four $\ion{H}{2}$ regions in M51 are 2-3 times higher than in the Orion Barphotodissociation region (PDR), with implications for the processesresponsible for the formation and evolution of hydrocarbonnanoparticles, including PAHs.  The deuteration of the aliphaticmaterial is found to anticorrelate with helium ionization in theassociated $\ion{H}{2}$ , suggesting that harsh FUV radiation may act tolower the deuteration of aliphatics in PDRs near massive stars.  Noevidence is found for deuteration of aromatic material, with $({\rm D/H})_{\rm arom} \ltsim 0.016$ : deuteration of the aliphatic materialexceeds that of the aromatic material by at least a factor 10.  Theobserved levels of deuteration may account for the depletion of Dobserved in the Galactic interstellar medium.  If so, the $4.65\micron$ feature may be detectable in absorption.

</div>

<div id="div_fig1">

<img src="tmp_2504.02538/./f2.png" alt="Fig2" width="100%"/>

**Figure 2. -** $\footnote$size$4-5\micron$ spectra
  ($R\approx 1100$) of four star-forming regions in M 51 (see text).
  Observed wavelengths have been corrected for redshift (see Table
  \ref{tab:params} for adopted radial velocities).  Three of the
  spectra show $CO_2$ ice absorption (strongest in E-Strip
  $\ion${H}{2}).  The E-Strip $\ion${H}{2} spectrum also shows
  4.56-4.69$\micron$ CO ice absorption.  A dust emission feature near
  $4.65\micron$ is present in all spectra.
   (*fig:m51_4-5*)

</div>
<div id="div_fig2">

<img src="tmp_2504.02538/./f9a.png" alt="Fig9.1" width="50%"/><img src="tmp_2504.02538/./f9b.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** $\footnote$size(a) $F(4.65)/F(3.4)$
  vs. $F(3.4)/F(3.29)$, where $F(4.65)$ is the power in the
  $4.65\micron$ aliphatic C-D feature, $F(3.4)$ is the "clipped"
  estimate of the non-aromatic C-H stretch power, and $F(3.29)$ is the
  "clipped" estimate of the aromatic C-H stretch power (see Figure
  \ref{fig:CH}).  Right-hand scale shows estimated $(\Da/\Ha)_\aliph$
  in the emitting nanoparticles, assuming $K/f=1.2$.  Deuteration is
  2-3 times higher in M51 than in the Orion Bar.  $(\Da/\Ha)_\aliph$
  does not appear to be correlated with nonaromatic fraction.  (b)
  Same as (a), but plotted against $\He^+/\Ha^+$.  $(\Da/\Ha)_\aliph$
  appears to depend on the hardness of the radiation ionizing the
  $\ion${H}{2} region associated with the PDR where the PAHs are
  located.   (*fig:cdr*)

</div>
<div id="div_fig3">

<img src="tmp_2504.02538/./f10a.png" alt="Fig10.1" width="25%"/><img src="tmp_2504.02538/./f10b.png" alt="Fig10.2" width="25%"/><img src="tmp_2504.02538/./f10c.png" alt="Fig10.3" width="25%"/><img src="tmp_2504.02538/./f10d.png" alt="Fig10.4" width="25%"/>

**Figure 10. -** $\footnote$size$4.2-4.6\micron$ spectra
  of the four sightlines in Figure \ref{fig:m51_4-5}.  Black curves:
  observed spectra.  Blue dashed curves: estimate of continuum without
  ice absorption.  Green curves: continuum with ice absorption
  applied.  Red curves: continuum plus emission lines of H, He, and
  $H_2$(see text).  The expected wavelengths for nitrile CN
  ($\sim$4.35$\mu$m) and aromatic C--D ($\sim$4.40$\mu$m) are
  indicated.  In the M51 spectra there is no evidence of nitrile CN
  emission or aromatic C--D emission; the inset profile shows our
  estimated upper limit on the strength of either feature. (*fig:m51aromCD*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.02538"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

426  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

18  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
